# **SETUP**

*Install* DuckDB

In [ ]:
!pip install duckdb

Import libraries

In [ ]:
import pandas as pd
import duckdb
import time

Download:
*   [NYC Taxi Dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page)
*   Weather Dataset



In [ ]:
!wget "https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2016-01.csv"
!wget "https://pdet.github.io/assets/data/weather.csv"

In [ ]:
import requests

requests.get("https://pdet.github.io/assets/data/weather.csv")
requests.get("https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2016-01.csv")

# **Load the data** 

To Load the data in Pandas, we can use the read_csv function.

In [ ]:
pandas_taxi = pd.read_csv("https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2016-01.csv")
pandas_weather = pd.read_csv("https://pdet.github.io/assets/data/weather.csv")

There are two main ways of loading the data in DuckDB
1.   Through the [SQL API](https://duckdb.org/docs/sql/statements/create_table)


In [99]:
#Get a DuckDB Connection, pass it a string to store data on Disk.
# duck_conn = duckdb.connect("path.db")
duck_conn.close()
# Use the create table syntax to read data from a CSV File
# load_cab_trip_sql = 'SELECT * FROM pandas_taxi'
# load_weather_sql = 'SELECT * FROM pandas_weather'
# duck_conn.execute(load_cab_trip_sql)
# duck_conn.execute(load_weather_sql)

In [ ]:
duck_conn.fetchone()

2.   Through the [Relational API](https://github.com/cwida/duckdb/blob/master/examples/python/duckdb-python.py)


In [ ]:
# Use the from_csv_auto function to point to the table
duck_rel_taxi = duck_conn.from_df(pandas_taxi)
# print(duck_rel_taxi)

# duck_rel_weather = duck_conn.from_df(pandas_weather)
# print(duck_rel_weather)

#Note that by using the Relational API the data is not yet stored in a table, 
# so future access to it will read the CSV files again.

#Use the table function to get a Python Relational object that references a table.
# This will allow us to use the relational API on a table.
# duck_taxi = duck_conn.table(duck_rel_taxi)
# duck_weather = duck_conn.table('weather')

duck_conn.execute("CREATE TABLE cab_trip AS SELECT * FROM pandas_taxi") # Create a table from the pandas_taxi dataframe 

In [ ]:
duck_conn.execute("SHOW TABLES").df()

In [ ]:
duck_conn.execute("DESCRIBE cab_trip").df()
# duck_conn.execute("DESCRIBE cab_trip").fetchall()
# duck_conn.execute("DESCRIBE cab_trip").fetchnumpy()

In [ ]:
duck_conn.execute("SELECT * FROM cab_trip LIMIT 10").df()

In [ ]:
duck_conn.execute("SUMMARIZE cab_trip").df()

# **Queries** 

# **Q1 (Simple Aggregation)** 
**Average of** (passenger_count, trip_distance, fare_amount e total_amount.)


In [ ]:
# Fill SQL with query
sql = """ """

start_time = time.monotonic()
result = pandas_taxi.agg(
    passenger_count=('passenger_count', 'mean'),
    trip_distance=('trip_distance', 'mean'),
    fare_amount=('fare_amount', 'mean'),
    total_amount=('total_amount', 'mean')
  )
print ("Pandas: " + str(time.monotonic() - start_time))

start_time = time.monotonic()
res = duck_conn.execute(sql).fetchall()
print ("DuckDB: " + str(time.monotonic() - start_time))

duck_conn.execute('PRAGMA threads=2')

start_time = time.monotonic()
res = duck_conn.execute(sql).fetchall()
print ("DuckDB (2T): " + str(time.monotonic() - start_time))

duck_conn.execute('PRAGMA threads=1')

# TIP: Use aggregate function of duck_taxi
start_time = time.monotonic()
res = ...
print ("DuckDB (RAPI): " + str(time.monotonic() - start_time))

# **Q2 (Aggregation + Group By)**
**Average of** (passenger_count, trip_distance, fare_amount e total_amount.)

**Grouped by** passenger_count

In [ ]:
# Fill SQL with query
sql = """ """

start_time = time.monotonic()
result = pandas_taxi.groupby(['passenger_count']).agg(
    trip_distance=('trip_distance', 'mean'),
    fare_amount=('fare_amount', 'mean'),
    total_amount=('total_amount', 'mean')
  )
print ("Pandas: " + str(time.monotonic() - start_time))

start_time = time.monotonic()
res = duck_conn.execute(sql).fetchall()
print ("DuckDB: " + str(time.monotonic() - start_time))

duck_conn.execute('PRAGMA threads=2')

start_time = time.monotonic()
res = duck_conn.execute(sql).fetchall()
print ("DuckDB (2T): " + str(time.monotonic() - start_time))

duck_conn.execute('PRAGMA threads=1')

start_time = time.monotonic()
# TIP: Use aggregate function of duck_taxi
res = ...
print ("DuckDB (RAPI): " + str(time.monotonic() - start_time))

# **Q3 (Aggregation + Group By + Filter)**
**Average of** (passenger_count, trip_distance, fare_amount e total_amount.)

**Grouped by** passenger_count

**Filter** total_amount < 10

In [ ]:
# Fill SQL with query
sql = """  """

start_time = time.monotonic()
filtered_df = pandas_taxi[pandas_taxi['total_amount'] < 10]
result = filtered_df.groupby(['passenger_count']).agg(
    trip_distance=('trip_distance', 'mean'),
    fare_amount=('fare_amount', 'mean'),
    total_amount=('total_amount', 'mean')
  )
print ("Pandas: " + str(time.monotonic() - start_time))

start_time = time.monotonic()
res = duck_conn.execute(sql).fetchall()
print ("DuckDB: " + str(time.monotonic() - start_time))

duck_conn.execute('PRAGMA threads=2')

start_time = time.monotonic()
res = duck_conn.execute(sql).fetchall()
print ("DuckDB (2T): " + str(time.monotonic() - start_time))

duck_conn.execute('PRAGMA threads=1')

start_time = time.monotonic()
# TIP: Use filter function of duck_taxi
duck_filter = ...
# TIP: Use aggregate function of duck_filter
res = ...
print ("DuckDB (RAPI): " + str(time.monotonic() - start_time))

# **Q4 (Aggregation + Group By + Filter + Join)**
**Average of** (passenger_count, trip_distance, fare_amount e total_amount.)

**Join** cab_trip.tpep_pickup_datetime::DATE = weather.date

**Grouped by** passenger_count

**Filter** total_amount < 10 and maximum_temperature < 50

In [ ]:
# Fill SQL with query (Note the cast in the join key cab_trip.tpep_pickup_datetime::DATE = weather.date )

sql = """  """

start_time = time.monotonic()

merged = pandas_taxi.merge(
  pandas_weather,
  left_on='tpep_pickup_datetime',
  right_on='date')

filtered_df = merged[merged['total_amount'] < 10]

filtered_df = filtered_df[filtered_df['maximum_temperature'] < 50 ]

result = filtered_df.groupby(['passenger_count']).agg(
    trip_distance=('trip_distance', 'mean'),
    fare_amount=('fare_amount', 'mean'),
    total_amount=('total_amount', 'mean')
  )

print ("Pandas: " + str(time.monotonic() - start_time))

start_time = time.monotonic()
res = duck_conn.execute(sql).fetchall()
print ("DuckDB: " + str(time.monotonic() - start_time))

duck_conn.execute('PRAGMA threads=2')

start_time = time.monotonic()
res = duck_conn.execute(sql).fetchall()
print ("DuckDB (2T): " + str(time.monotonic() - start_time))

duck_conn.execute('PRAGMA threads=1')

start_time = time.monotonic()
# TIP: Use filter function of duck_taxi
duck_taxi_filter = ...
# TIP: Use filter function of duck_weather
duck_weather_filter = ...
# TIP: Use join function of duck_taxi_filter with duck_weather_filter
duck_join = ...
# TIP: Use aggregate function
res = ...

print ("DuckDB (RAPI): " + str(time.monotonic() - start_time))

# **Transactions** 

Let's say that no car in NYC can fit more than 5 passengers.
Consider all trips with more than 5 passengers as dirty, and remove them.


In [ ]:
print(duck_conn.execute('select count(*) from cab_trip where passenger_count > 6').fetchall())

In [100]:
# import sqlite3

# sqlite_conn = sqlite3.connect('taxi.db')
sqlite_conn.close()
# pandas_taxi.to_sql('cap_trip_sqlite', sqlite_conn, if_exists='replace')


In [ ]:
print(sqlite_conn.execute('select count(*) from cap_trip_sqlite where passenger_count > 6').fetchall())

In [ ]:
res = duck_conn.execute('select count(*) from cab_trip where passenger_count > 6').fetchall()

start_time = time.monotonic()
result = pandas_taxi[pandas_taxi['total_amount'] < 6]
result.to_csv('result.csv.tmp', sep=',')
print ("Pandas: " + str(time.monotonic() - start_time))

start_time = time.monotonic()
# TIP: Use delete from SQL
res = ...
print ("DuckDB: " + str(time.monotonic() - start_time))

In [ ]:
res = sqlite_conn.execute('select count(*) from cap_trip_sqlite where passenger_count > 6').fetchall()

start_time = time.monotonic()
result = pandas_taxi[pandas_taxi['total_amount'] < 6]
result.to_csv('result.csv.tmp', sep=',')
print ("Pandas: " + str(time.monotonic() - start_time))

start_time = time.monotonic()
# TIP: Use delete from SQL
res = ...
print ("SQLiteDB: " + str(time.monotonic() - start_time))